In [752]:
import numpy as np
import pandas as pd

In [753]:
submission = pd.read_csv("SampleSubmission.csv")
submission[['year','month','site_code','product_code']] = submission.ID.str.split(" X ",expand=True,)

submission = submission.drop(['ID', 'prediction'], axis=1).set_index('site_code')

site_data = pd.read_csv("service_delivery_site_data.csv")
site = site_data[['site_code', 'site_region', 'site_district']].set_index('site_code')
site.columns = ['region', 'district']

submission = submission.join(site, how='left').reset_index()

In [754]:
train = pd.read_csv("Train.csv")

In [755]:
len(train)

35753

In [756]:
submission

,site_code,year,month,product_code,region,district
0,C1004,2019,7,AS27132,AGNEBY-TIASSA-ME,AGBOVILLE
1,C1004,2019,7,AS27000,AGNEBY-TIASSA-ME,AGBOVILLE
2,C1004,2019,7,AS27137,AGNEBY-TIASSA-ME,AGBOVILLE
3,C1004,2019,7,AS27138,AGNEBY-TIASSA-ME,AGBOVILLE
4,C1004,2019,7,AS27133,AGNEBY-TIASSA-ME,AGBOVILLE
...,...,...,...,...,...,...
3084,C5076,2019,9,AS27000,KABADOUGOU-BAFING-FOLON,KORO
3085,C5076,2019,9,AS27139,KABADOUGOU-BAFING-FOLON,KORO
3086,C5076,2019,9,AS27137,KABADOUGOU-BAFING-FOLON,KORO
3087,C5076,2019,9,AS27138,KABADOUGOU-BAFING-FOLON,KORO


In [757]:
merged = pd.concat([train, submission])
merged

,year,month,region,district,site_code,product_code,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered
0,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27134,75.0,0.0,21.0,-54.0,0.0,18.0,0.0,100.0
1,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27132,3.0,0.0,3.0,0.0,0.0,2.0,0.0,10.0
2,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27000,0.0,75.0,22.0,0.0,53.0,9.0,0.0,0.0
3,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27137,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,2019,1,INDENIE-DJUABLIN,ABENGOUROU,C4001,AS27138,45.0,0.0,2.0,0.0,43.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3084,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3085,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3086,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3087,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [758]:
product = pd.read_csv("product.csv")

site = pd.read_csv('service_delivery_site_data.csv')

product_type = product[['product_code', 'product_type']].set_index('product_code')

merged = merged.set_index('product_code')

merged = merged.join(product_type, how='left').reset_index().set_index('site_code')

site_type = site[['site_code', 'site_type']].set_index('site_code')

merged = merged.join(site_type, how='left').reset_index().set_index('site_code').reset_index()

In [759]:
merged.year = merged.loc[:, 'year'].astype('int')
merged.month = merged.loc[:, 'month'].astype('int')

In [760]:
def make_date_block(row):
    return (row.year - 2016)*12 + (row.month)

In [761]:
merged['date_block_num'] = merged.apply(make_date_block, axis=1)

In [762]:
merged

,site_code,product_code,year,month,region,district,stock_initial,stock_received,stock_distributed,stock_adjustment,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,product_type,site_type,date_block_num
0,C1004,AS21126,2019,4,AGNEBY-TIASSA-ME,AGBOVILLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Injectable Contraceptive,Hospital,40
1,C1004,AS21126,2019,5,AGNEBY-TIASSA-ME,AGBOVILLE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,Injectable Contraceptive,Hospital,41
2,C1004,AS21126,2019,9,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Injectable Contraceptive,Hospital,45
3,C1004,AS27000,2019,1,AGNEBY-TIASSA-ME,AGBOVILLE,0.0,0.0,13.0,51.0,38.0,30.0,0.0,30.0,Oral Contraceptive (Pill),Hospital,37
4,C1004,AS27000,2019,2,AGNEBY-TIASSA-ME,AGBOVILLE,38.0,0.0,19.0,0.0,19.0,31.0,0.0,43.0,Oral Contraceptive (Pill),Hospital,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38837,C5076,AS27132,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,45
38838,C5076,AS27133,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Injectable Contraceptive,Hospital,45
38839,C5076,AS27137,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contraceptive Implant,Hospital,45
38840,C5076,AS27138,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contraceptive Implant,Hospital,45


In [763]:
def lag_feature( df,lags, cols ):
    for col in cols:
        print(col)
        tmp = df[[ "date_block_num", "site_code", "product_code",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "site_code", "product_code", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','site_code','product_code'], how='left')
    return df

In [764]:
#Stock_distributed for a contraceptive product at a service delivery site is reported in a given month as '0', even though stock was available (stock_initial was above 0) and stock_distributed is historically well above 0.

merged = lag_feature(merged, [1,2,3], ['stock_distributed'])

stock_distributed


In [765]:
#Spurious zeros: https://zindi.africa/competitions/usaids-intelligent-forecasting-challenge-model-future-contraceptive-use/data
#Exclude emergency contraceptive, Female condom and IUD where the average is low
#Excluding when stock_initial + stock_adjustment + stock_received = 0 (or in other words when stock_end = 0)
mask1 = merged['stock_distributed'] == 0
#mask2A = merged['stock_distributed_lag_1'] > 5
#mask2B = (merged['stock_distributed_lag_1'] > 0) & (merged['stock_distributed_lag_2'] > 0)
mask2 = (merged['stock_distributed_lag_1'] > 5) & (merged['stock_distributed_lag_2'] > 5) & (merged['stock_distributed_lag_3'] > 5)
mask3 = merged['stock_initial'] > 0
mask4 = ~(merged['product_code'].isin(['AS17005', 'AS27139', 'AS42018']))
mask5 = ~(merged['stock_end'] == 0)
fullmask = mask1 & mask2 & mask3 & mask4 & mask5
spurious_zeros = merged.loc[fullmask, :]
merged = merged.loc[~fullmask, :].reset_index().drop('index', axis=1) # dropping where fullmask is true

In [766]:
#Sanity check - stock_end from before should be stock initial, not true...
#cases when stock_end_lag_1 is actually > 0 but stock_initial is zero along with most other fields must be error
check = lag_feature(merged, [1], ['stock_end'])
check.stock_end_lag_1 = check.loc[:, 'stock_end_lag_1'].fillna(0)
mask1 = check['stock_end_lag_1'] != check['stock_initial'] #we need this to make sure that stock_initial being zero 
#(and hence nothing distributed) is a typo not actual data
mask2 = (spurious_zeros2['stock_end_lag_1'] > 0) & (spurious_zeros2['stock_initial'] == 0) & (spurious_zeros2['stock_received'] == 0) & (spurious_zeros2['stock_distributed'] == 0) & (spurious_zeros2['stock_adjustment'] == 0) & (spurious_zeros2['stock_end'] == 0) & (spurious_zeros2['stock_stockout_days'] == 0)
mask3 = ~((merged.date_block_num == 43) | (merged.date_block_num == 44) | (merged.date_block_num == 45)) # not test set
fullmask = mask1 & mask2 & mask3
spurious_zeros2 = check.loc[fullmask,:]
merged = merged.loc[~fullmask, :].reset_index().drop('index', axis=1) # dropping where fullmask is true

stock_end


In [767]:
group = merged.groupby( ["site_code"] ).agg({"date_block_num" : ["min"]})
group.columns = ["first_introduced_site"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["site_code"], how = "left")

In [768]:
group = merged.groupby( ["product_code"] ).agg({"date_block_num" : ["min"]})
group.columns = ["first_introduced_product"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["product_code"], how = "left")

In [769]:
group = merged.groupby( ["site_code", "product_code"] ).agg({"date_block_num" : ["min"]})
group.columns = ["first_introduced_product_site"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["site_code", "product_code"], how = "left")

In [770]:
merged['month_since_introduced_site'] = merged['date_block_num'] - merged['first_introduced_site']
merged['month_since_introduced_product'] = merged['date_block_num'] - merged['first_introduced_product']
merged['month_since_introduced_product_site'] = merged['date_block_num'] - merged['first_introduced_product_site']

merged = merged.drop(['first_introduced_site', 'first_introduced_product', 'first_introduced_product_site'], axis=1)

# merged['new_site'] = False
# mask = merged.first_introduced_site == merged.date_block_num
# merged.loc[mask, 'new_site'] = True

# merged['new_product'] = False
# mask = merged.first_introduced_product == merged.date_block_num
# merged.loc[mask, 'new_product'] = True

In [771]:
merged.columns

Index(['site_code', 'product_code', 'year', 'month', 'region', 'district',
       'stock_initial', 'stock_received', 'stock_distributed',
       'stock_adjustment', 'stock_end', 'average_monthly_consumption',
       'stock_stockout_days', 'stock_ordered', 'product_type', 'site_type',
       'date_block_num', 'stock_distributed_lag_1', 'stock_distributed_lag_2',
       'stock_distributed_lag_3', 'month_since_introduced_site',
       'month_since_introduced_product',
       'month_since_introduced_product_site'],
      dtype='object')

In [772]:
#there's not much we can do with stock_received and stock_adjusted and stock_initial; these are all random quantities
#missing from the test set
#we will instead lag stock_end and stock_ordered
merged = merged.drop(['stock_initial', 'stock_received', 'stock_adjustment'], axis=1)
#we will also drop stock_distributed, we will recreate them (now less samples, removed spurious zeros)
merged = merged.drop(['stock_distributed_lag_1', 'stock_distributed_lag_2', 'stock_distributed_lag_3'], axis=1)

In [773]:
#sanity check - should be 3089 long
mask = (merged.date_block_num == 43) | (merged.date_block_num == 44) | (merged.date_block_num == 45)
merged.loc[mask,:]

,site_code,product_code,year,month,region,district,stock_distributed,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,product_type,site_type,date_block_num,month_since_introduced_site,month_since_introduced_product,month_since_introduced_product_site
2,C1004,AS21126,2019,9,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,Injectable Contraceptive,Hospital,45,38,43,5
28,C1004,AS27000,2019,7,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,43,36,42,36
29,C1004,AS27000,2019,8,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,44,37,43,37
30,C1004,AS27000,2019,9,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,45,38,44,38
55,C1004,AS27132,2019,7,AGNEBY-TIASSA-ME,AGBOVILLE,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,43,36,42,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38503,C5076,AS27132,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,Oral Contraceptive (Pill),Hospital,45,0,44,0
38504,C5076,AS27133,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,Injectable Contraceptive,Hospital,45,0,44,0
38505,C5076,AS27137,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,Contraceptive Implant,Hospital,45,0,44,0
38506,C5076,AS27138,2019,9,KABADOUGOU-BAFING-FOLON,KORO,NaN,NaN,NaN,NaN,NaN,Contraceptive Implant,Hospital,45,0,44,0


In [774]:
merged.to_csv('cleaned_data.csv', index=False)

In [637]:
#Feature Engineering

In [869]:
merged = pd.read_csv('cleaned_data.csv')
validation = False #True

In [870]:
mask = (merged.date_block_num == 43) | (merged.date_block_num == 44) | (merged.date_block_num == 45)
merged.loc[~mask,:].isna().any()

site_code                              False
product_code                           False
year                                   False
month                                  False
region                                 False
district                               False
stock_distributed                      False
stock_end                              False
average_monthly_consumption            False
stock_stockout_days                    False
stock_ordered                           True
product_type                           False
site_type                              False
date_block_num                         False
month_since_introduced_site            False
month_since_introduced_product         False
month_since_introduced_product_site    False
dtype: bool

In [871]:
#rename
merged['outcome'] = merged['stock_distributed']
merged = merged.drop('stock_distributed', axis=1)

In [872]:
if validation:
    train_mask = merged.date_block_num < 40
    test_mask = (merged.date_block_num == 40) | (merged.date_block_num == 41) | (merged.date_block_num == 42)
else:
    train_mask = merged.date_block_num < 43
    test_mask = (merged.date_block_num == 43) | (merged.date_block_num == 44) | (merged.date_block_num == 45)

In [873]:
#making sure that validation is as similar to test set as possible, .i.e we don't know stock_end, stock_ordered, stock_stockout_days
#and obviously outcome
if validation:
    validation_outcome = merged.loc[test_mask, 'outcome']
    merged.loc[test_mask,['stock_end', 'stock_ordered', 'average_monthly_consumption', 'stock_stockout_days', 'outcome']] = np.nan

In [874]:
sum(validation_outcome.isna())

0

In [875]:
merged.loc[train_mask,:].isna().any()

site_code                              False
product_code                           False
year                                   False
month                                  False
region                                 False
district                               False
stock_end                              False
average_monthly_consumption            False
stock_stockout_days                    False
stock_ordered                           True
product_type                           False
site_type                              False
date_block_num                         False
month_since_introduced_site            False
month_since_introduced_product         False
month_since_introduced_product_site    False
outcome                                False
dtype: bool

In [876]:
merged_train = merged.loc[train_mask,:]
merged_train.loc[merged_train.stock_ordered.isna(), :] #fine, catboost will take care of it

,site_code,product_code,year,month,region,district,stock_end,average_monthly_consumption,stock_stockout_days,stock_ordered,product_type,site_type,date_block_num,month_since_introduced_site,month_since_introduced_product,month_since_introduced_product_site,outcome
169,C1007,AS17005,2019,1,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,0.0,0.0,0.0,NaN,Female Condom,University Hospital/National Institute,37,35,36,33,0.0
181,C1007,AS17005,2018,7,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,0.0,0.0,0.0,NaN,Female Condom,University Hospital/National Institute,31,29,30,27,0.0
209,C1007,AS27000,2018,7,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,590.0,69.0,0.0,NaN,Oral Contraceptive (Pill),University Hospital/National Institute,31,29,30,27,36.0
285,C1007,AS27133,2018,7,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,58.0,6.0,0.0,NaN,Injectable Contraceptive,University Hospital/National Institute,31,29,30,29,15.0
328,C1007,AS27134,2018,7,ABIDJAN 1-GRANDS PONTS,YOPOUGON-OUEST-SONGON,119.0,25.0,0.0,NaN,Injectable Contraceptive,University Hospital/National Institute,31,29,30,29,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38383,C5066,AS27132,2017,5,KABADOUGOU-BAFING-FOLON,MINIGNAN,0.0,0.0,0.0,NaN,Oral Contraceptive (Pill),Hospital,17,0,16,0,0.0
38418,C5066,AS27134,2017,5,KABADOUGOU-BAFING-FOLON,MINIGNAN,0.0,0.0,0.0,NaN,Injectable Contraceptive,Hospital,17,0,16,0,0.0
38442,C5066,AS27137,2017,5,KABADOUGOU-BAFING-FOLON,MINIGNAN,0.0,0.0,0.0,NaN,Contraceptive Implant,Hospital,17,0,16,0,0.0
38482,C5066,AS27139,2017,5,KABADOUGOU-BAFING-FOLON,MINIGNAN,0.0,0.0,0.0,NaN,Emergency Contraceptive (Pill),Hospital,17,0,16,0,0.0


In [877]:
merged.columns

Index(['site_code', 'product_code', 'year', 'month', 'region', 'district',
       'stock_end', 'average_monthly_consumption', 'stock_stockout_days',
       'stock_ordered', 'product_type', 'site_type', 'date_block_num',
       'month_since_introduced_site', 'month_since_introduced_product',
       'month_since_introduced_product_site', 'outcome'],
      dtype='object')

In [878]:
#first lag and then mean-encode is taken care of by lagging single features and catboost
#first mean-encode and then lag: we're doing it

In [879]:
group = merged.loc[train_mask,:].groupby( ["date_block_num"] ).agg({"outcome" : ["mean"]})
group.columns = ["date_avg_outcome"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["date_block_num"], how = "left")

In [880]:
group = merged.loc[train_mask,:].groupby( ["date_block_num", "product_code"] ).agg({"outcome" : ["mean"]})
group.columns = ["date_product_avg_outcome"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["date_block_num", "product_code"], how = "left")

In [881]:
group = merged.loc[train_mask,:].groupby( ["date_block_num", "site_code"] ).agg({"outcome" : ["mean"]})
group.columns = ["date_site_avg_outcome"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["date_block_num", "site_code"], how = "left")

In [882]:
# group = merged.loc[train_mask,:].groupby( ["date_block_num", "district"] ).agg({"outcome" : ["mean"]})
# group.columns = ["date_district_avg_outcome"]
# group.reset_index(inplace = True)

# merged = pd.merge(merged, group, on = ["date_block_num", "district"], how = "left")

In [883]:
group = merged.loc[train_mask,:].groupby( ["date_block_num", "region"] ).agg({"outcome" : ["mean"]})
group.columns = ["date_region_avg_outcome"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["date_block_num", "region"], how = "left")

In [884]:
group = merged.loc[train_mask,:].groupby( ["date_block_num", "region", "product_code"] ).agg({"outcome" : ["mean"]})
group.columns = ["date_region_product_avg_outcome"]
group.reset_index(inplace = True)

merged = pd.merge(merged, group, on = ["date_block_num", "region", "product_code"], how = "left")

In [885]:
#create further features: pct fulfilled; pct used
#maybe first lag by stock_distributed and get rid of spurious zeros

In [886]:
matrix = lag_feature(merged, [1,2,3], ['stock_end', 'stock_ordered', 'average_monthly_consumption', 'stock_stockout_days', 'outcome', 'date_avg_outcome', 'date_product_avg_outcome', 'date_site_avg_outcome', 'date_region_avg_outcome', 'date_region_product_avg_outcome']) #try with 12

stock_end
stock_ordered
average_monthly_consumption
stock_stockout_days
outcome
date_avg_outcome
date_product_avg_outcome
date_site_avg_outcome
date_region_avg_outcome
date_region_product_avg_outcome


In [887]:
#Need to remove features that only make sense lagged, i.e. we don't know in advance
#we keep outcome, since we will need it for model evaluation
#we will remove date_block_num later: it can't help with testing, since that's how we partition (new values)
matrix = matrix.drop(['stock_end', 'stock_ordered', 'average_monthly_consumption', 'stock_stockout_days', 'date_avg_outcome', 'date_product_avg_outcome', 'date_site_avg_outcome', 'date_region_avg_outcome', 'date_region_product_avg_outcome'], axis=1)

In [888]:
matrix.columns

Index(['site_code', 'product_code', 'year', 'month', 'region', 'district',
       'product_type', 'site_type', 'date_block_num',
       'month_since_introduced_site', 'month_since_introduced_product',
       'month_since_introduced_product_site', 'outcome', 'stock_end_lag_1',
       'stock_end_lag_2', 'stock_end_lag_3', 'stock_ordered_lag_1',
       'stock_ordered_lag_2', 'stock_ordered_lag_3',
       'average_monthly_consumption_lag_1',
       'average_monthly_consumption_lag_2',
       'average_monthly_consumption_lag_3', 'stock_stockout_days_lag_1',
       'stock_stockout_days_lag_2', 'stock_stockout_days_lag_3',
       'outcome_lag_1', 'outcome_lag_2', 'outcome_lag_3',
       'date_avg_outcome_lag_1', 'date_avg_outcome_lag_2',
       'date_avg_outcome_lag_3', 'date_product_avg_outcome_lag_1',
       'date_product_avg_outcome_lag_2', 'date_product_avg_outcome_lag_3',
       'date_site_avg_outcome_lag_1', 'date_site_avg_outcome_lag_2',
       'date_site_avg_outcome_lag_3', 'date_regi

In [889]:
matrix

,site_code,product_code,year,month,region,district,product_type,site_type,date_block_num,month_since_introduced_site,month_since_introduced_product,month_since_introduced_product_site,outcome,stock_end_lag_1,stock_end_lag_2,stock_end_lag_3,stock_ordered_lag_1,stock_ordered_lag_2,stock_ordered_lag_3,average_monthly_consumption_lag_1,average_monthly_consumption_lag_2,average_monthly_consumption_lag_3,stock_stockout_days_lag_1,stock_stockout_days_lag_2,stock_stockout_days_lag_3,outcome_lag_1,outcome_lag_2,outcome_lag_3,date_avg_outcome_lag_1,date_avg_outcome_lag_2,date_avg_outcome_lag_3,date_product_avg_outcome_lag_1,date_product_avg_outcome_lag_2,date_product_avg_outcome_lag_3,date_site_avg_outcome_lag_1,date_site_avg_outcome_lag_2,date_site_avg_outcome_lag_3,date_region_avg_outcome_lag_1,date_region_avg_outcome_lag_2,date_region_avg_outcome_lag_3,date_region_product_avg_outcome_lag_1,date_region_product_avg_outcome_lag_2,date_region_product_avg_outcome_lag_3
0,C1004,AS21126,2019,4,AGNEBY-TIASSA-ME,AGBOVILLE,Injectable Contraceptive,Hospital,40,33,38,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C1004,AS21126,2019,5,AGNEBY-TIASSA-ME,AGBOVILLE,Injectable Contraceptive,Hospital,41,34,39,1,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,16.202258,NaN,NaN,2.157303,NaN,NaN,25.250000,NaN,NaN,14.415730,NaN,NaN,0.000000,NaN,NaN
2,C1004,AS21126,2019,9,AGNEBY-TIASSA-ME,AGBOVILLE,Injectable Contraceptive,Hospital,45,38,43,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C1004,AS27000,2019,1,AGNEBY-TIASSA-ME,AGBOVILLE,Oral Contraceptive (Pill),Hospital,37,30,36,30,13.0,0.0,60.0,13.0,0.0,51.0,64.0,44.0,37.0,32.0,0.0,0.0,0.0,60.0,17.0,55.0,15.271530,14.017745,17.060606,28.070423,25.112676,29.801418,25.428571,11.714286,17.666667,21.257143,13.808824,13.211268,11.272727,5.100000,17.909091
4,C1004,AS27000,2019,2,AGNEBY-TIASSA-ME,AGBOVILLE,Oral Contraceptive (Pill),Hospital,38,31,37,31,19.0,38.0,0.0,60.0,30.0,0.0,51.0,30.0,44.0,37.0,0.0,0.0,0.0,13.0,60.0,17.0,15.678926,15.271530,14.017745,28.280822,28.070423,25.112676,15.428571,25.428571,11.714286,11.408451,21.257143,13.808824,16.909091,11.272727,5.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38503,C5076,AS27132,2019,9,KABADOUGOU-BAFING-FOLON,KORO,Oral Contraceptive (Pill),Hospital,45,0,44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38504,C5076,AS27133,2019,9,KABADOUGOU-BAFING-FOLON,KORO,Injectable Contraceptive,Hospital,45,0,44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38505,C5076,AS27137,2019,9,KABADOUGOU-BAFING-FOLON,KORO,Contraceptive Implant,Hospital,45,0,44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38506,C5076,AS27138,2019,9,KABADOUGOU-BAFING-FOLON,KORO,Contraceptive Implant,Hospital,45,0,44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [890]:
if validation:
    matrix.loc[train_mask,:].to_csv('matrix_train.csv', index=False)
    matrix.loc[test_mask,:].to_csv('matrix_validation.csv', index=False)
    validation_outcome.to_csv('validation_outcome.csv', index=False)
else:
    matrix.loc[train_mask,:].to_csv('train.csv', index=False)
    matrix.loc[test_mask,:].to_csv('test.csv', index=False)

In [844]:
#Example - calc is fine, NaN happens because product exits and enters site
pd.set_option('display.max_columns', 100)
mask = (matrix['site_code'] == 'C4056') & (matrix['product_code'] == 'AS27132')
matrix.loc[mask, :].sort_values('date_block_num')

,site_code,product_code,year,month,region,district,product_type,site_type,date_block_num,month_since_introduced_site,month_since_introduced_product,month_since_introduced_product_site,outcome,stock_end_lag_1,stock_end_lag_2,stock_end_lag_3,stock_ordered_lag_1,stock_ordered_lag_2,stock_ordered_lag_3,average_monthly_consumption_lag_1,average_monthly_consumption_lag_2,average_monthly_consumption_lag_3,ran_out_lag_1,ran_out_lag_2,ran_out_lag_3,outcome_lag_1,outcome_lag_2,outcome_lag_3,date_avg_outcome_lag_1,date_avg_outcome_lag_2,date_avg_outcome_lag_3,date_product_avg_outcome_lag_1,date_product_avg_outcome_lag_2,date_product_avg_outcome_lag_3,date_site_avg_outcome_lag_1,date_site_avg_outcome_lag_2,date_site_avg_outcome_lag_3,date_region_avg_outcome_lag_1,date_region_avg_outcome_lag_2,date_region_avg_outcome_lag_3,date_region_product_avg_outcome_lag_1,date_region_product_avg_outcome_lag_2,date_region_product_avg_outcome_lag_3
34671,C4056,AS27132,2018,7,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,31,29,30,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34672,C4056,AS27132,2018,8,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,32,30,31,1,0.0,0.0,NaN,NaN,20.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,15.332611,NaN,NaN,5.230088,NaN,NaN,4.500000,NaN,NaN,7.411765,NaN,NaN,0.666667,NaN,NaN
34673,C4056,AS27132,2018,9,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,33,31,32,2,0.0,0.0,0.0,NaN,0.0,20.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,14.127932,15.332611,NaN,3.760684,5.230088,NaN,2.166667,4.500000,NaN,9.000000,7.411765,NaN,0.333333,0.666667,NaN
34674,C4056,AS27132,2018,10,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,34,32,33,3,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.539446,14.127932,15.332611,3.613445,3.760684,5.230088,8.333333,2.166667,4.500000,9.941176,9.000000,7.411765,0.666667,0.333333,0.666667
34675,C4056,AS27132,2018,11,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,35,33,34,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.060606,16.539446,14.127932,5.163934,3.613445,3.760684,10.000000,8.333333,2.166667,19.294118,9.941176,9.000000,1.666667,0.666667,0.333333
34666,C4056,AS27132,2019,2,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,38,36,37,7,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,14.017745,NaN,NaN,5.844262,NaN,NaN,7.166667,NaN,NaN,14.058824,NaN,NaN,0.666667
34667,C4056,AS27132,2019,3,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,39,37,38,8,0.0,0.0,NaN,NaN,20.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,13.861723,NaN,NaN,5.269841,NaN,NaN,6.333333,NaN,NaN,12.705882,NaN,NaN,0.000000,NaN,NaN
34668,C4056,AS27132,2019,4,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,40,38,39,9,NaN,0.0,0.0,NaN,5.0,20.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,14.821000,13.861723,NaN,1.214286,5.269841,NaN,8.166667,6.333333,NaN,13.117647,12.705882,NaN,0.000000,0.000000,NaN
34669,C4056,AS27132,2019,5,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,41,39,40,10,NaN,NaN,0.0,0.0,NaN,5.0,20.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,14.821000,13.861723,NaN,1.214286,5.269841,NaN,8.166667,6.333333,NaN,13.117647,12.705882,NaN,0.000000,0.000000
34670,C4056,AS27132,2019,6,INDENIE-DJUABLIN,BETTIE,Oral Contraceptive (Pill),Hospital,42,40,41,11,NaN,NaN,NaN,0.0,NaN,NaN,5.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,14.821000,NaN,NaN,1.214286,NaN,NaN,8.166667,NaN,NaN,13.117647,NaN,NaN,0.000000
